# High Resolution inference 

In [2]:
# run in the root of the repository
%load_ext autoreload
%autoreload 2
 
%cd ../..

/private/home/pfz/09-videoseal/videoseal-dev


/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from videoseal.utils.display import save_img
from videoseal.utils import Timer
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.evals.metrics import bit_accuracy, psnr, ssim
from videoseal.augmentation import Identity, JPEG
from videoseal.modules.jnd import JND, VarianceBasedJND

import os
import omegaconf
from tqdm import tqdm
import gc
from PIL import Image

import torch
import torchvision

to_tensor = torchvision.transforms.ToTensor()
to_pil = torchvision.transforms.ToPILImage()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu" 

/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Directory containing videos
assets_dir = "/checkpoint/pfz/projects/videoseal/assets/imgs"
# assets_dir = "/private/home/pfz/_images"
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

# Checkpoint
ckpts = {
    # "videoseal0.1": '/private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.5_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=unet_small2/checkpoint.pth',
    # "videoseal0.2b": "/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1111-videoseal0.2-archsearch-4nodes/_attenuation=None_nbits=64_finetune_detector_start=800_embedder_model=unet_small2_quant/checkpoint.pth",
    # "videoseal0.2a": "/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1109-videoseal0.2-discloss-fix-hing-sleepwake-4nodes/_scaling_w=0.5_lambda_i=0.5_disc_hinge_on_logits_fake=True_sleepwake=False_video_start=500/checkpoint.pth",
    # "videoseal0.4": "/large_experiments/meres/hadyelsahar/2024_logs/1120-videoseal0.4/_scaling_w=0.5_sleepwake=False_videowam_step_size=4_extractor_model=sam_tiny/checkpoint.pth",
    # "trustmark": "baseline/trustmark",
    # "wam": "baseline/wam",
    # "cin": "baseline/cin",
    # "mbrs": "baseline/mbrs",
    "ychannel": "/checkpoint/pfz/2025_logs/0115_vseal_rgb_96bits_nopercep_yuv/_scaling_w=0.05_lambda_d=0.5_extractor_model=sam_small/checkpoint.pth",
}

for ckpt_name, ckpt_path in ckpts.items():

    # a timer to measure the time
    timer = Timer()

    # Iterate over all checkpoints
    wam = setup_model_from_checkpoint(ckpt_path)
    wam.eval()
    wam.to(device)

    # Iterate over all video files in the directory
    files = [f for f in os.listdir(assets_dir) if f.endswith(".png")]
    files = [os.path.join(assets_dir, f) for f in files]

    for file in tqdm(files, desc=f"Processing Images"):
        # load image
        imgs = Image.open(file, "r").convert("RGB")  # keep only rgb channels
        imgs = to_tensor(imgs).unsqueeze(0).float()

        # Watermark embedding
        timer.start()
        outputs = wam.embed(imgs, is_video=False)
        torch.cuda.synchronize()
        # print(f"embedding watermark  - took {timer.stop():.2f}s")

        # compute diff
        imgs_w = outputs["imgs_w"]  # b c h w
        msgs = outputs["msgs"]  # b k
        diff = imgs_w - imgs

        # save
        timer.start()
        base_save_name = os.path.join(output_dir, os.path.basename(file).replace(".png", ""))
        # print(f"saving videos to {base_save_name}")
        save_img(imgs[0], f"{base_save_name}_ori.png")
        save_img(imgs_w[0], f"{base_save_name}_wm.png")
        save_img(20*diff[0].abs(), f"{base_save_name}_diff.png")

        # Compute min and max values, reshape, and normalize
        min_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).min(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        max_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).max(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        normalized_images = (diff - min_vals) / (max_vals - min_vals)

        # Save the normalized video
        save_img(normalized_images[0], f"{base_save_name}_diff_norm.png")
        # print(f"saving videos - took {timer.stop():.2f}s")

        # Metrics
        imgs_aug = imgs_w
        outputs = wam.detect(imgs_aug, is_video=False)
        metrics = {
            "bit_accuracy": bit_accuracy(
                outputs["preds"][:, 1:],
                msgs
            ).nanmean().item(),
            "psnr": psnr(imgs_w, imgs).item(),
            "ssim": ssim(imgs_w, imgs).item()
        }

        # Augment video
        # print(f"compressing and detecting watermarks")
        for qf in [80, 40]:
            imgs_aug, _ = JPEG()(imgs_w, None,qf)

            # detect
            timer.start()
            outputs = wam.detect(imgs_aug, is_video=True)
            preds = outputs["preds"]
            # print(preds)
            bit_preds = preds[:, 1:]  # b k ...
            bit_accuracy_ = bit_accuracy(
                bit_preds,
                msgs
            ).nanmean().item()
            
            metrics[f"bit_accuracy_qf{qf}"] = bit_accuracy_

        print(metrics)

        del outputs, imgs, imgs_w, diff, min_vals, max_vals, normalized_images

    # Free model from GPU
    del wam
    torch.cuda.empty_cache()

Model loaded successfully from /checkpoint/pfz/2025_logs/0115_vseal_rgb_96bits_nopercep_yuv/_scaling_w=0.05_lambda_d=0.5_extractor_model=sam_small/checkpoint.pth with message: _IncompatibleKeys(missing_keys=['detector.pixel_decoder.linear.bias'], unexpected_keys=[])


Processing Images:  25%|██▌       | 1/4 [00:01<00:05,  1.80s/it]

{'bit_accuracy': 1.0, 'psnr': 33.734886169433594, 'ssim': 0.975898027420044, 'bit_accuracy_qf80': 1.0, 'bit_accuracy_qf40': 1.0}


Processing Images:  50%|█████     | 2/4 [00:02<00:01,  1.03it/s]

{'bit_accuracy': 1.0, 'psnr': 34.71742630004883, 'ssim': 0.9345903396606445, 'bit_accuracy_qf80': 1.0, 'bit_accuracy_qf40': 1.0}


Processing Images:  75%|███████▌  | 3/4 [00:07<00:02,  2.77s/it]

{'bit_accuracy': 1.0, 'psnr': 32.39744567871094, 'ssim': 0.9686586260795593, 'bit_accuracy_qf80': 1.0, 'bit_accuracy_qf40': 1.0}


Processing Images: 100%|██████████| 4/4 [00:07<00:00,  1.93s/it]

{'bit_accuracy': 1.0, 'psnr': 33.420494079589844, 'ssim': 0.9584458470344543, 'bit_accuracy_qf80': 1.0, 'bit_accuracy_qf40': 1.0}


## With attenuation

In [12]:
# Directory containing videos
assets_dir = "/checkpoint/pfz/projects/videoseal/assets/imgs"
output_dir = "outputs"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Checkpoint
ckpt_path = "videoseal"
# ckpt_path = "/checkpoint/pfz/2025_logs/0115_vseal_rgb_96bits_nopercep_yuv/_scaling_w=0.05_lambda_d=0.5_extractor_model=sam_small/checkpoint.pth"

# a timer to measure the time
timer = Timer()

# Iterate over all checkpoints
wam = setup_model_from_checkpoint(ckpt_path)
wam.eval()
wam.to(device)

# create attenuation
attenuation = VarianceBasedJND(
    mode="variance",
    max_variance_value_for_clipping=300,
    min_heatmap_value=0.1,
    avg_pool_kernel_size=3
)
wam.attenuation = attenuation
wam.blender.scaling_w = 20.0

# Iterate over all video files in the directory
files = [f for f in os.listdir(assets_dir) if f.endswith(".png")]
files = [os.path.join(assets_dir, f) for f in files]

for file in tqdm(files, desc=f"Processing Images"):
    # load image
    imgs = Image.open(file, "r").convert("RGB")  # keep only rgb channels
    imgs = to_tensor(imgs).unsqueeze(0).float()

    # Watermark embedding
    timer.start()
    outputs = wam.embed(imgs, is_video=False)
    # torch.cuda.synchronize()
    # print(f"embedding watermark  - took {timer.stop():.2f}s")

    # compute diff
    imgs_w = outputs["imgs_w"]  # b c h w
    msgs = outputs["msgs"]  # b k
    diff = imgs_w - imgs

    # save
    timer.start()
    base_save_name = os.path.join(output_dir, os.path.basename(file).replace(".png", ""))
    save_img(imgs[0], f"{base_save_name}_ori.png")
    save_img(imgs_w[0], f"{base_save_name}_wm.png")
    save_img(diff[0], f"{base_save_name}_diff.png")

    # Compute min and max values, reshape, and normalize
    min_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).min(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
    max_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).max(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
    normalized_images = (diff - min_vals) / (max_vals - min_vals)

    # Save the normalized video
    save_img(normalized_images[0], f"{base_save_name}_diff_norm.png")

    # Metrics
    imgs_aug = imgs_w
    outputs = wam.detect(imgs_aug, is_video=False)
    metrics = {
        "bit_accuracy": bit_accuracy(
            outputs["preds"][:, 1:],
            msgs
        ).nanmean().item(),
        "psnr": psnr(imgs_w, imgs).item(),
        "ssim": ssim(imgs_w, imgs).item()
    }

    # Augment video
    for qf in [80, 40]:
        imgs_aug, _ = JPEG()(imgs_w, None,qf)

        # detect
        timer.start()
        outputs = wam.detect(imgs_aug, is_video=False)
        preds = outputs["preds"]
        bit_preds = preds[:, 1:]  # b k ...
        bit_accuracy_ = bit_accuracy(
            bit_preds,
            msgs
        ).nanmean().item()
        metrics[f"bit_accuracy_qf_{qf}"] = bit_accuracy_
    
    print(metrics)
    del outputs, imgs, imgs_w, diff, min_vals, max_vals, normalized_images

# Free model from GPU
del wam
torch.cuda.empty_cache()

Model loaded successfully from /private/home/pfz/.cache/huggingface/hub/models--facebook--video_seal/snapshots/8037ef59ba2b2ec8fb8b55298ff37b8ccddd078d/checkpoint.pth with message: <All keys matched successfully>


Processing Images:  25%|██▌       | 1/4 [00:01<00:03,  1.26s/it]

{'bit_accuracy': 0.9166666865348816, 'psnr': 46.27751541137695, 'ssim': 0.9988591074943542, 'bit_accuracy_qf_80': 0.9270833134651184, 'bit_accuracy_qf_40': 0.8854166865348816}


Processing Images:  50%|█████     | 2/4 [00:01<00:01,  1.39it/s]

{'bit_accuracy': 0.9895833134651184, 'psnr': 49.10234069824219, 'ssim': 0.9982087016105652, 'bit_accuracy_qf_80': 0.9895833134651184, 'bit_accuracy_qf_40': 0.7395833134651184}


Processing Images:  75%|███████▌  | 3/4 [00:06<00:02,  2.43s/it]

{'bit_accuracy': 0.9895833134651184, 'psnr': 45.21706008911133, 'ssim': 0.9980943202972412, 'bit_accuracy_qf_80': 0.9791666865348816, 'bit_accuracy_qf_40': 0.9895833134651184}


Processing Images: 100%|██████████| 4/4 [00:06<00:00,  1.65s/it]

{'bit_accuracy': 1.0, 'psnr': 46.6027717590332, 'ssim': 0.998041570186615, 'bit_accuracy_qf_80': 0.9895833134651184, 'bit_accuracy_qf_40': 0.96875}
